In [3]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,cross_val_score,KFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import Pipeline

from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation,LeakyReLU
from keras.layers.core import Dense,Dropout
from keras.optimizers import Adam,SGD
from keras.metrics import categorical_crossentropy,binary_crossentropy, sparse_categorical_crossentropy
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
table = pd.read_csv('\\Users\\vatss\\codes\\Ronaldo\\data.csv')
Input = table.loc[np.isnan(table['is_goal']) != True]
prdInput = table.loc[np.isnan(table['is_goal']) == True]
Input.drop(columns=["home/away",'type_of_shot','type_of_combined_shot','date_of_game', 'remaining_sec.1', 'team_name', 'remaining_sec', 'knockout_match', 'team_id', 'match_event_id', 'Unnamed: 0', 'match_id', 'game_season', 'location_x', 'lat/lng', 'power_of_shot.1', 'remaining_min.1', 'knockout_match.1', 'shot_id_number'
],inplace=True)

C:\Users\vatss\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Cleaning

In [5]:
numerical=['power_of_shot','distance_of_shot',"distance_of_shot.1","location_y","remaining_min"]
non_numerical=['area_of_shot','shot_basics','range_of_shot']
for col in non_numerical:
    temp = pd.DataFrame(data={col:Input[col]})
    mode = temp.describe()[col]['top']
    temp.fillna(mode,inplace=True)
    Input[col]=temp[col].values
for col in numerical:
    temp = pd.DataFrame(data={col:Input[col]})
    mean=temp[col].mean()
    temp.fillna(mean,inplace=True)
    Input[col]=temp[col].values

C:\Users\vatss\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\vatss\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [6]:
yData = Input['is_goal']
xData = Input.drop(columns='is_goal')

# Getting data in integer form

In [7]:
for col in non_numerical:
    temp = pd.get_dummies(xData[col])
    xData.drop(columns=[col],inplace=True)
    xData = pd.concat([xData.reset_index(drop=True),temp.reset_index(drop=True)], axis=1)

# Scaling

In [8]:
scaler = preprocessing.StandardScaler()
for col in numerical:
    scaler.fit(np.array(xData[col]).reshape(-1,1))
    xData[col]=scaler.transform(np.array(xData[col]).reshape(-1,1))

In [15]:
xData.describe()

,location_y,remaining_min,power_of_shot,distance_of_shot,distance_of_shot.1,Center(C),Left Side Center(LC),Left Side(L),Mid Ground(MG),Right Side Center(RC),...,Left Corner,Mid Ground Line,Mid Range,Penalty Spot,Right Corner,16-24 ft.,24+ ft.,8-16 ft.,Back Court Shot,Less Than 8 ft.
count,2.442900e+04,2.442900e+04,2.442900e+04,2.442900e+04,2.442900e+04,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000,...,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000,24429.000000
mean,2.098126e-16,-6.086253e-16,-4.842825e-17,1.046997e-15,5.647144e-16,0.464980,0.124074,0.116583,0.002906,0.148021,...,0.008842,0.002333,0.440910,0.173769,0.012362,0.254615,0.194482,0.207786,0.002865,0.340251
std,1.000020e+00,1.000020e+00,1.000020e+00,1.000020e+00,1.000020e+00,0.498782,0.329673,0.320929,0.053834,0.355128,...,0.093617,0.048249,0.496506,0.378918,0.110499,0.435654,0.395809,0.405731,0.053454,0.473803
min,-1.579410e+00,-1.450557e+00,-1.353223e+00,-1.474173e+00,-1.614149e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.847752e-01,-8.561936e-01,-4.638244e-01,-8.173870e-01,-6.464010e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-8.699364e-02,0.000000e+00,-3.949724e-16,5.832709e-02,-9.654436e-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.524904e-01,9.268968e-01,4.255744e-01,7.151126e-01,2.883553e-01,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,8.156274e+00,1.818442e+00,3.983170e+00,7.173504e+00,4.232367e+00,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
X_train,X_test,y_train,y_test = train_test_split(xData,yData,random_state=0)

In [10]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

## MultiLayer Perceptron

In [75]:
from sklearn.neural_network import MLPClassifier
clf=MLPClassifier(validation_fraction=0.3,alpha=0.01,activation='relu',batch_size=20,hidden_layer_sizes=(150,75,60))
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=20, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(150, 75, 60), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.3, verbose=False, warm_start=False)

In [76]:
clf.score(X_test,y_test)

0.5920104780615586

# function for evaluation

In [76]:
def eval_model():
    model = Sequential()
    model.add(Dense(37,input_shape=(23,),kernel_regularizer=regularizers.l2(),kernel_initializer='uniform',activation='selu'))
    model.add(Dense(77,kernel_regularizer=regularizers.l2(),activation='selu',kernel_initializer='uniform'))
    model.add(Dense(2, activation='sigmoid',kernel_regularizer=regularizers.l2()))
    model.compile(optimizer=Adam(lr=0.01),loss=binary_crossentropy,metrics=['accuracy'])
    return model

In [77]:
model = KerasClassifier(build_fn=eval_model, epochs=10, batch_size=30, verbose=0)
results = cross_val_score(model, X_train, y_train, cv=5)
print("Mean:",results.mean()*100,"%")

Mean: 59.52721953392028 %


### 1. n1=256 n2=71
### 2. n1=256 n2=61
### 3. n1=270 n2=61 lr=0.012
### 3. n1=270 n2=65 lr=0.01
### 2. n1=220 n2=65
#### 30 60

# function for Hyperparameterization

In [106]:
def create_model(n1=10,n2=15,n3=20,n4=50,n5=40,n6=10,optimizer='Adam',activation='relu',init_mode='uniform'):
    model = Sequential()
    model.add(Dense(n1,input_shape=(23,),kernel_regularizer=regularizers.l2(),kernel_initializer=init_mode,activation=activation))
    model.add(Dense(n2,kernel_regularizer=regularizers.l2(),activation=activation,kernel_initializer=init_mode))
    model.add(Dense(n3,kernel_regularizer=regularizers.l2(),activation=activation,kernel_initializer=init_mode))
    model.add(Dense(n4,kernel_regularizer=regularizers.l2(),activation=activation,kernel_initializer=init_mode))
    model.add(Dense(n5,kernel_regularizer=regularizers.l2(),activation=activation,kernel_initializer=init_mode))
    model.add(Dense(n6,kernel_regularizer=regularizers.l2(),activation=activation,kernel_initializer=init_mode))
    model.add(Dense(2, activation='sigmoid',kernel_regularizer=regularizers.l2()))
    model.compile(optimizer=optimizer,loss=binary_crossentropy,metrics=['accuracy'])
    return model

In [111]:
from keras.utils.generic_utils import get_custom_objects
model = KerasClassifier(build_fn=create_model, verbose=2)
batch_size=[10,20,50,100,150,200]
epoch=[5,10,15,20,25,30,35,40]
get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})
optimizer=['sgd','adam','adagrad']
activation=['relu','elu','selu']
init_mode = ['uniform', 'lecun_uniform','glorot_normal', 'glorot_uniform']
n1 = np.arange(30,75,5)
n2 = np.arange(20,75,5)
n3 = np.arange(10,70,5)
n4 = np.arange(1,50,5)
n5 = np.arange(1,10,1)
n6 = np.arange(1,10,1)
param_grid = dict(n1=n1,n2=n2,n3=n3,n4=n4,n5=n5,n6=n6,init_mode=init_mode,batch_size=batch_size,epochs=epoch,optimizer=optimizer,activation=activation)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,n_jobs=-1, cv=4)
grid.fit(X_train, y_train)

Epoch 1/5
 - 1s - loss: 1.5674 - accuracy: 0.5972
Epoch 2/5
 - 0s - loss: 1.0470 - accuracy: 0.6070
Epoch 3/5
 - 0s - loss: 0.8869 - accuracy: 0.6080
Epoch 4/5
 - 0s - loss: 0.8134 - accuracy: 0.6098
Epoch 5/5
 - 0s - loss: 0.7743 - accuracy: 0.6092


RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001E2C0F7AA48>,
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'activation': ['relu', 'elu', 'selu'],
                                        'batch_size': [10, 20, 50, 100, 150,
                                                       200],
                                        'epochs': [5, 10, 15, 20, 25, 30, 35,
                                                   40],
                                        'init_mode': ['uniform',
                                                      'lecun_uniform',
                                                      'glorot_normal',...
                                        'n1': array([30, 35, 40, 45, 50, 55, 60, 65, 70]),
                                        'n2': array([20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]),
                                 

In [116]:
print(grid.best_params_)
Model = grid.best_estimator_.model
print("accuracy",grid.best_estimator_.model.evaluate(X_test,y_test,batch_size=200)[1])

{'optimizer': 'adagrad', 'n6': 6, 'n5': 4, 'n4': 41, 'n3': 35, 'n2': 35, 'n1': 70, 'init_mode': 'glorot_normal', 'epochs': 5, 'batch_size': 200, 'activation': 'selu'}
6108/6108 [==============================] - 0s 6us/step
accuracy 0.5921741724014282


In [113]:
for layer in Model.layers:
    print(layer.activation)

<function selu at 0x000001E2D15FA5E8>
<function selu at 0x000001E2D15FA5E8>
<function selu at 0x000001E2D15FA5E8>
<function selu at 0x000001E2D15FA5E8>
<function selu at 0x000001E2D15FA5E8>
<function selu at 0x000001E2D15FA5E8>
<function sigmoid at 0x000001E2D15FA8B8>


In [29]:
import random 
max=[0,""]
for au1 in range(1,100,5):
    for au2 in range(1,70,5):
        model = Sequential()
        model.add(Dense(au1,input_shape=(20,),kernel_regularizer=regularizers.l2()))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dense(au2,kernel_regularizer=regularizers.l2()))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dense(2,kernel_regularizer=regularizers.l2(),activation='sigmoid'))
        model.compile(Adam(lr=0.005),loss=binary_crossentropy,metrics=['accuracy'])
        model.fit(X_train,y_train,batch_size=200,epochs=40,verbose=2,validation_split=0.2,shuffle=True)
        sc = model.evaluate(X_test,y_test_binary,batch_size=100)[1]
        if sc > max[0]:
            max[0] = sc
            max[1] = str(au1)+" "+str(au2)

Train on 11317 samples, validate on 2830 samples
Epoch 1/40
 - 1s - loss: 0.7718 - accuracy: 0.6007 - val_loss: 0.7143 - val_accuracy: 0.6025
Epoch 2/40
 - 0s - loss: 0.7009 - accuracy: 0.5996 - val_loss: 0.6912 - val_accuracy: 0.6141
Epoch 3/40
 - 0s - loss: 0.6903 - accuracy: 0.6077 - val_loss: 0.6878 - val_accuracy: 0.6141
Epoch 4/40
 - 0s - loss: 0.6886 - accuracy: 0.6071 - val_loss: 0.6874 - val_accuracy: 0.6078
Epoch 5/40
 - 0s - loss: 0.6884 - accuracy: 0.6071 - val_loss: 0.6870 - val_accuracy: 0.6141
Epoch 6/40
 - 0s - loss: 0.6880 - accuracy: 0.6063 - val_loss: 0.6863 - val_accuracy: 0.6141
Epoch 7/40
 - 0s - loss: 0.6878 - accuracy: 0.6022 - val_loss: 0.6873 - val_accuracy: 0.6025
Epoch 8/40
 - 0s - loss: 0.6877 - accuracy: 0.6014 - val_loss: 0.6867 - val_accuracy: 0.6025
Epoch 9/40
 - 0s - loss: 0.6871 - accuracy: 0.6048 - val_loss: 0.6861 - val_accuracy: 0.6141
Epoch 10/40
 - 0s - loss: 0.6874 - accuracy: 0.6051 - val_loss: 0.6863 - val_accuracy: 0.6025
Epoch 11/40
 - 0s - 

Epoch 6/40
 - 0s - loss: 0.6887 - accuracy: 0.6040 - val_loss: 0.6866 - val_accuracy: 0.6141
Epoch 7/40
 - 0s - loss: 0.6880 - accuracy: 0.6060 - val_loss: 0.6864 - val_accuracy: 0.6141
Epoch 8/40
 - 0s - loss: 0.6877 - accuracy: 0.6059 - val_loss: 0.6861 - val_accuracy: 0.6141
Epoch 9/40
 - 0s - loss: 0.6876 - accuracy: 0.6062 - val_loss: 0.6860 - val_accuracy: 0.6141
Epoch 10/40
 - 0s - loss: 0.6876 - accuracy: 0.6052 - val_loss: 0.6861 - val_accuracy: 0.6141
Epoch 11/40
 - 0s - loss: 0.6874 - accuracy: 0.6037 - val_loss: 0.6856 - val_accuracy: 0.6141
Epoch 12/40
 - 0s - loss: 0.6870 - accuracy: 0.6069 - val_loss: 0.6862 - val_accuracy: 0.6110
Epoch 13/40
 - 0s - loss: 0.6874 - accuracy: 0.6043 - val_loss: 0.6860 - val_accuracy: 0.6141
Epoch 14/40
 - 0s - loss: 0.6876 - accuracy: 0.6028 - val_loss: 0.6869 - val_accuracy: 0.6141
Epoch 15/40
 - 0s - loss: 0.6872 - accuracy: 0.6047 - val_loss: 0.6856 - val_accuracy: 0.6025
Epoch 16/40
 - 0s - loss: 0.6870 - accuracy: 0.6047 - val_loss: 

Epoch 12/40
 - 0s - loss: 0.6869 - accuracy: 0.6061 - val_loss: 0.6856 - val_accuracy: 0.6025
Epoch 13/40
 - 0s - loss: 0.6869 - accuracy: 0.6048 - val_loss: 0.6867 - val_accuracy: 0.6025
Epoch 14/40
 - 0s - loss: 0.6869 - accuracy: 0.6024 - val_loss: 0.6858 - val_accuracy: 0.6025
Epoch 15/40
 - 0s - loss: 0.6871 - accuracy: 0.6041 - val_loss: 0.6855 - val_accuracy: 0.6141
Epoch 16/40
 - 0s - loss: 0.6872 - accuracy: 0.6055 - val_loss: 0.6854 - val_accuracy: 0.6141
Epoch 17/40
 - 0s - loss: 0.6868 - accuracy: 0.6055 - val_loss: 0.6867 - val_accuracy: 0.6025
Epoch 18/40
 - 0s - loss: 0.6871 - accuracy: 0.5950 - val_loss: 0.6850 - val_accuracy: 0.6141
Epoch 19/40
 - 0s - loss: 0.6864 - accuracy: 0.6035 - val_loss: 0.6852 - val_accuracy: 0.6141
Epoch 20/40
 - 0s - loss: 0.6867 - accuracy: 0.6034 - val_loss: 0.6858 - val_accuracy: 0.6141
Epoch 21/40
 - 0s - loss: 0.6870 - accuracy: 0.6048 - val_loss: 0.6866 - val_accuracy: 0.6025
Epoch 22/40
 - 0s - loss: 0.6862 - accuracy: 0.6040 - val_lo

Epoch 18/40
 - 0s - loss: 0.6868 - accuracy: 0.6070 - val_loss: 0.6860 - val_accuracy: 0.6021
Epoch 19/40
 - 0s - loss: 0.6868 - accuracy: 0.6040 - val_loss: 0.6852 - val_accuracy: 0.6110
Epoch 20/40
 - 0s - loss: 0.6864 - accuracy: 0.6034 - val_loss: 0.6847 - val_accuracy: 0.6141
Epoch 21/40
 - 0s - loss: 0.6864 - accuracy: 0.6034 - val_loss: 0.6849 - val_accuracy: 0.6141
Epoch 22/40
 - 0s - loss: 0.6859 - accuracy: 0.6041 - val_loss: 0.6847 - val_accuracy: 0.6120
Epoch 23/40
 - 0s - loss: 0.6858 - accuracy: 0.6028 - val_loss: 0.6843 - val_accuracy: 0.6141
Epoch 24/40
 - 0s - loss: 0.6863 - accuracy: 0.6020 - val_loss: 0.6845 - val_accuracy: 0.6141
Epoch 25/40
 - 0s - loss: 0.6861 - accuracy: 0.6040 - val_loss: 0.6901 - val_accuracy: 0.6000
Epoch 26/40
 - 0s - loss: 0.6859 - accuracy: 0.6019 - val_loss: 0.6847 - val_accuracy: 0.6141
Epoch 27/40
 - 0s - loss: 0.6858 - accuracy: 0.6049 - val_loss: 0.6844 - val_accuracy: 0.6120
Epoch 28/40
 - 0s - loss: 0.6871 - accuracy: 0.6033 - val_lo

Epoch 24/40
 - 0s - loss: 0.6863 - accuracy: 0.6052 - val_loss: 0.6848 - val_accuracy: 0.6021
Epoch 25/40
 - 0s - loss: 0.6860 - accuracy: 0.6030 - val_loss: 0.6851 - val_accuracy: 0.6025
Epoch 26/40
 - 0s - loss: 0.6863 - accuracy: 0.6050 - val_loss: 0.6844 - val_accuracy: 0.6141
Epoch 27/40
 - 0s - loss: 0.6859 - accuracy: 0.6051 - val_loss: 0.6854 - val_accuracy: 0.6025
Epoch 28/40
 - 0s - loss: 0.6860 - accuracy: 0.6030 - val_loss: 0.6845 - val_accuracy: 0.6141
Epoch 29/40
 - 0s - loss: 0.6853 - accuracy: 0.6009 - val_loss: 0.6888 - val_accuracy: 0.6021
Epoch 30/40
 - 0s - loss: 0.6860 - accuracy: 0.6033 - val_loss: 0.6847 - val_accuracy: 0.6021
Epoch 31/40
 - 0s - loss: 0.6860 - accuracy: 0.6023 - val_loss: 0.6844 - val_accuracy: 0.6141
Epoch 32/40
 - 0s - loss: 0.6861 - accuracy: 0.6025 - val_loss: 0.6852 - val_accuracy: 0.6025
Epoch 33/40
 - 0s - loss: 0.6855 - accuracy: 0.6040 - val_loss: 0.6842 - val_accuracy: 0.6141
Epoch 34/40
 - 0s - loss: 0.6858 - accuracy: 0.6048 - val_lo

...

Epoch 22/40
 - 0s - loss: 0.6863 - accuracy: 0.6040 - val_loss: 0.6853 - val_accuracy: 0.6025
Epoch 23/40
 - 0s - loss: 0.6864 - accuracy: 0.6019 - val_loss: 0.6857 - val_accuracy: 0.6141
Epoch 24/40
 - 0s - loss: 0.6861 - accuracy: 0.6046 - val_loss: 0.6846 - val_accuracy: 0.6141
Epoch 25/40
 - 0s - loss: 0.6867 - accuracy: 0.6063 - val_loss: 0.6862 - val_accuracy: 0.6025
Epoch 26/40
 - 0s - loss: 0.6863 - accuracy: 0.6042 - val_loss: 0.6847 - val_accuracy: 0.6021
Epoch 27/40
 - 0s - loss: 0.6859 - accuracy: 0.6033 - val_loss: 0.6856 - val_accuracy: 0.6141
Epoch 28/40
 - 0s - loss: 0.6862 - accuracy: 0.6034 - val_loss: 0.6848 - val_accuracy: 0.6141
Epoch 29/40
 - 0s - loss: 0.6859 - accuracy: 0.6061 - val_loss: 0.6850 - val_accuracy: 0.6078
Epoch 30/40
 - 0s - loss: 0.6857 - accuracy: 0.6071 - val_loss: 0.6848 - val_accuracy: 0.6025
Epoch 31/40
 - 0s - loss: 0.6858 - accuracy: 0.6019 - val_loss: 0.6847 - val_accuracy: 0.6141
Epoch 32/40
 - 0s - loss: 0.6857 - accuracy: 0.6056 - val_lo

 - 0s - loss: 0.6860 - accuracy: 0.6021 - val_loss: 0.6842 - val_accuracy: 0.6141
Epoch 28/40
 - 0s - loss: 0.6863 - accuracy: 0.6051 - val_loss: 0.6846 - val_accuracy: 0.6141
Epoch 29/40
 - 0s - loss: 0.6871 - accuracy: 0.6047 - val_loss: 0.6848 - val_accuracy: 0.6028
Epoch 30/40
 - 0s - loss: 0.6859 - accuracy: 0.6040 - val_loss: 0.6860 - val_accuracy: 0.6025
Epoch 31/40
 - 0s - loss: 0.6870 - accuracy: 0.6023 - val_loss: 0.6842 - val_accuracy: 0.6141
Epoch 32/40
 - 0s - loss: 0.6866 - accuracy: 0.6002 - val_loss: 0.6846 - val_accuracy: 0.6141
Epoch 33/40
 - 0s - loss: 0.6861 - accuracy: 0.5969 - val_loss: 0.6847 - val_accuracy: 0.6025
Epoch 34/40
 - 0s - loss: 0.6857 - accuracy: 0.6017 - val_loss: 0.6859 - val_accuracy: 0.6025
Epoch 35/40
 - 0s - loss: 0.6864 - accuracy: 0.6017 - val_loss: 0.6852 - val_accuracy: 0.6025
Epoch 36/40
 - 0s - loss: 0.6858 - accuracy: 0.6016 - val_loss: 0.6850 - val_accuracy: 0.6025
Epoch 37/40
 - 0s - loss: 0.6858 - accuracy: 0.6030 - val_loss: 0.6842 -

Epoch 33/40
 - 0s - loss: 0.6860 - accuracy: 0.6058 - val_loss: 0.6892 - val_accuracy: 0.6000
Epoch 34/40
 - 0s - loss: 0.6863 - accuracy: 0.6047 - val_loss: 0.6848 - val_accuracy: 0.6141
Epoch 35/40
 - 0s - loss: 0.6866 - accuracy: 0.6039 - val_loss: 0.6857 - val_accuracy: 0.6025
Epoch 36/40
 - 0s - loss: 0.6858 - accuracy: 0.6015 - val_loss: 0.6842 - val_accuracy: 0.6141
Epoch 37/40
 - 0s - loss: 0.6859 - accuracy: 0.6030 - val_loss: 0.6844 - val_accuracy: 0.6141
Epoch 38/40
 - 0s - loss: 0.6853 - accuracy: 0.6069 - val_loss: 0.6857 - val_accuracy: 0.6141
Epoch 39/40
 - 0s - loss: 0.6857 - accuracy: 0.6023 - val_loss: 0.6842 - val_accuracy: 0.6141
Epoch 40/40
 - 0s - loss: 0.6857 - accuracy: 0.6035 - val_loss: 0.6845 - val_accuracy: 0.6060
4716/4716 [==============================] - 0s 7us/step
Train on 11317 samples, validate on 2830 samples
Epoch 1/40
 - 0s - loss: 0.8811 - accuracy: 0.5978 - val_loss: 0.6921 - val_accuracy: 0.5587
Epoch 2/40
 - 0s - loss: 0.6910 - accuracy: 0.578

 - 0s - loss: 0.6860 - accuracy: 0.6038 - val_loss: 0.6843 - val_accuracy: 0.6021
Epoch 39/40
 - 0s - loss: 0.6854 - accuracy: 0.6026 - val_loss: 0.6849 - val_accuracy: 0.6138
Epoch 40/40
 - 0s - loss: 0.6859 - accuracy: 0.5982 - val_loss: 0.6841 - val_accuracy: 0.6141
4716/4716 [==============================] - 0s 7us/step
Train on 11317 samples, validate on 2830 samples
Epoch 1/40
 - 0s - loss: 0.9011 - accuracy: 0.5704 - val_loss: 0.6916 - val_accuracy: 0.6141
Epoch 2/40
 - 0s - loss: 0.6906 - accuracy: 0.5723 - val_loss: 0.6870 - val_accuracy: 0.6141
Epoch 3/40
 - 0s - loss: 0.6893 - accuracy: 0.5979 - val_loss: 0.6865 - val_accuracy: 0.6141
Epoch 4/40
 - 0s - loss: 0.6894 - accuracy: 0.5926 - val_loss: 0.6872 - val_accuracy: 0.6141
Epoch 5/40
 - 0s - loss: 0.6886 - accuracy: 0.6046 - val_loss: 0.6874 - val_accuracy: 0.6141
Epoch 6/40
 - 0s - loss: 0.6886 - accuracy: 0.6054 - val_loss: 0.6863 - val_accuracy: 0.6141
Epoch 7/40
 - 0s - loss: 0.6885 - accuracy: 0.6048 - val_loss: 0.6

Epoch 3/40
 - 0s - loss: 0.6900 - accuracy: 0.5972 - val_loss: 0.6873 - val_accuracy: 0.6141
Epoch 4/40
 - 0s - loss: 0.6887 - accuracy: 0.6021 - val_loss: 0.6884 - val_accuracy: 0.6025
Epoch 5/40
 - 0s - loss: 0.6890 - accuracy: 0.6027 - val_loss: 0.6880 - val_accuracy: 0.6141
Epoch 6/40
 - 0s - loss: 0.6884 - accuracy: 0.5944 - val_loss: 0.6874 - val_accuracy: 0.6141
Epoch 7/40
 - 0s - loss: 0.6894 - accuracy: 0.5905 - val_loss: 0.6871 - val_accuracy: 0.5841
Epoch 8/40
 - 0s - loss: 0.6888 - accuracy: 0.5968 - val_loss: 0.6870 - val_accuracy: 0.6025
Epoch 9/40
 - 0s - loss: 0.6884 - accuracy: 0.6056 - val_loss: 0.6878 - val_accuracy: 0.6141
Epoch 10/40
 - 0s - loss: 0.6889 - accuracy: 0.6009 - val_loss: 0.6873 - val_accuracy: 0.5993
Epoch 11/40
 - 0s - loss: 0.6882 - accuracy: 0.5966 - val_loss: 0.6873 - val_accuracy: 0.6141
Epoch 12/40
 - 0s - loss: 0.6881 - accuracy: 0.5979 - val_loss: 0.6863 - val_accuracy: 0.6028
Epoch 13/40
 - 0s - loss: 0.6876 - accuracy: 0.5993 - val_loss: 0.6

In [30]:
max

[0.5969041585922241, '10 15']

In [178]:
print(model.metrics_names)


['loss', 'accuracy']
4716/4716 [==============================] - 0s 6us/step


[0.685267783252742, 0.5644614100456238]

In [38]:
clf = SVC(kernel="rbf")
grid = {"gamma":[0.00001,0.0001,0.001,0.01,1,10],"C":[0.1,1,10,100,0.001]}
obj = GridSearchCV(clf,grid)

In [39]:
obj.fit(X_train,y_train)
obj.best_estimator_

/home/shivam/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [42]:
clf = SVC(gamma=0.01, C=1, kernel="rbf")
clf.fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.5969041560644615


In [144]:
logisticReg = linear_model.LogisticRegression()
logisticReg.fit(X_train,y_train)
print(logisticReg.score(X_test,y_test))

0.6480470390592188


/home/shivam/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [145]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
print(rfc.score(X_test,y_test))

0.6157076858462831


/home/shivam/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
